# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    # Write your code here
    total_invested = P['Amount Invested'].sum()
    P['Expected Return'] = rf + P['Beta'] * mkt_rp
    P['Weighted Return'] = P['Expected Return'] * (P['Amount Invested'] / total_invested)
    portfolio_return = P['Weighted Return'].sum()

    return portfolio_return

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [8]:
def volatility(P, cor):
    # Write your code here
    n = len(P)
    weights = P['Amount Invested'] / P['Amount Invested'].sum()
    volatilities = P['Volatility']
    
    cor_matrix = np.full((n, n), cor)
    np.fill_diagonal(cor_matrix, 1)

    port_vol = np.sqrt(np.dot(weights, np.dot(np.outer(volatilities, volatilities) * cor_matrix, weights)))

    
    return port_vol

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [11]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    # Write your code here
    beta = vol / mkt_vol
    expected_return = rf + beta * mkt_rp
    
    return expected_return

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [13]:
def CAPM_beta(return_path, factors_path):
    #Write your code here
    df = pd.read_csv(return_path)
    df['month'] = df['month'].astype(str).str.zfill(2)
    df['year'] = df['year'].astype(str)
    df['date'] = df['year'] + df['month']
    df['date'] = df['date'].astype(int)

    df1 = pd.read_csv(factors_path)
    df1['Mkt-RF'] = df1['Mkt-RF'] / 100
    df1['RF'] = df1['RF'] / 100

    df_merged = pd.merge(df, df1, on='date')

    expected_returns = df_merged.groupby('CUSIP')['RET'].mean().reset_index(name='EXP_RETURN')

    df_merged['excess_return_firm'] = df_merged['RET'] - df_merged['RF']
    df_merged['excess_return_market'] = df_merged['Mkt-RF']

    grouped = df_merged.groupby('CUSIP')
    betas = grouped.apply(lambda x: np.cov(x['excess_return_firm'], x['excess_return_market'])[0, 1] / 
                          np.var(x['excess_return_market'])).reset_index(name='BETA')

    df_final = pd.merge(expected_returns, betas, on='CUSIP')
    
    
    
    output = pd.DataFrame(data = {
        "CUSIP": df_final['CUSIP'].values,
        "EXP_RETURN": df_final['EXP_RETURN'].values,
        "BETA": df_final['BETA'].values,
    })
    
    return output

In [14]:
CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.csv")

CUSIP  EXP_RETURN      BETA
0     00081T10    0.016252  2.603590
1     00108M10    0.011101  0.742466
2     00130H10    0.016152  1.460965
3     00163U10    0.015538  0.977903
4     00182C10    0.014772  1.412225
...        ...         ...       ...
3220   9959910    0.009933  0.765642
3221   9972410    0.012866  1.284197
3222   9972510    0.017177  1.070395
3223   9984910    0.009006  1.884472
3224   9985510    0.015699  1.171598

[3225 rows x 3 columns]

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [29]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    curve = np.polyfit(data['BETA'], data['EXP_RETURN'], 1)
    return curve[1]


In [30]:
slope(CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.csv"))

0.007986271615212929